# Objectives
* Create a Python object that we can use to manage state and simulate the behavior of Curve pools. We want to do this to:
    * Understand how perturbing pool parameters affects pool metrics 
        * e.g. For a pool with two assets I want to be observe the price impact on asset X when I add/subtract some amount of asset X. I want to produce the plots of reserves vs. price and delta_reserves vs. price.
    * Integrate with systems to import on-chain state into the data structures we define for modeling AMM pools. For example, I want this so I can use historical data Thrackle is gathering about Curve pools to contextualize IRL events with scenarios we want to simulate.

# Brainstorm: what is the right data structure for us to design as an abstract representation of an AMM pool? 

"Bad programmers worry about the code. Good programmers worry about data structures and their relationships." - Linus Torvalds

Let's start by thinking about the smart contract structure for a pool in:
1. Curve
2. Uniswap

The framework for each of these should be extensible to modeling AMM pools from other protocols too. 

## What is the data structure (abstract class) that best represents a generic AMM pool? 
* Observation: Most of our questions tend to revolve around understanding the state machine of a single pool. 
* Observation: There are state machines at several levels: consensus, protocol/app, pool.
    * Decision: It makes sense to start specific and then broaden out for this project. Thrackle already has several projects in flight to understand the protocol/app layer. We want to start by modeling a more narrow, granular system - namely an indivdual AMM pool.  

## Exercise TODO: enumerate the types of events relevant to Curve and Uniswap contracts. 
* How do we represent these events in our data structure? 
    * Which events are common to all AMMs and belong in the abstract class?
    * Which events have unique implementations in each AMM and belong in child classes (e.g. CurveCryptoPool(Pool)). 

In [30]:
from typing import Union, Optional, List
from abc import ABC, abstractmethod

class Pool(ABC): 
    
    '''
    Abstract class for a specific protocol's flavor of AMM pool.
    '''
    
    def __init__(self, 
                 token_symbols : List[str], 
                 token_prices : List[float],
                 token_balances : List[int] = None,
                 exchange_fee : float = 0.0) -> None:
        
        ### TODO: Think about __init__ pool state by reading on-chain data.
        ###       This is going to be child class-specific. What is interface? 
                  
        self.token_symbols = token_symbols
        self.token_prices = token_prices
        self.token_balances = token_balances
        
    def _get_token_id(self, symbol : str) -> int:
        return self.token_symbols.index(symbol)
    
    def __str__(self) -> str:
        return "\n".join([
            f"{s} has balance {b} at price {p}"
            for s, b, p in zip(self.token_symbols, self.token_balances, self.token_prices)
        ])
    
    @abstractmethod
    def add_liquidity(self, add_amounts : Union[int, List[int]]) -> None:
        raise NotImplementedError
    
    @abstractmethod
    def withdraw_liquidity(self, withdraw_amounts : Union[int, List[int]]) -> None:
        raise NotImplementedError
    
    @abstractmethod
    def exchange(self, 
                 sell_token_symbol : str,
                 buy_token_symbol : str,
                 sell_amount : int,
                 buy_amount : int) -> None:
        raise NotImplementedError
    
    @property
    def exchange_fee(self) -> float:
        return self._exchange_fee

    @exchange_fee.setter
    def exchange_fee(self, fee : float) -> None:
        if fee < 0:
            raise ValueError("Negative fees feels like questionable mechanism design...")
        self._exchange_fee = fee

In [37]:
from typing import Union, Optional, List

class FakePool(Pool):
    
    def __init__(self, 
                 token_symbols : List[str], 
                 token_prices : List[float],
                 token_balances : List[int] = None,
                 exchange_fee : float = 0.0,
                 pool_params : dict = {}) -> None:

        super().__init__(token_symbols, token_prices, token_balances, exchange_fee)
        self.pool_params = pool_params
        
    def add_liquidity(self, 
                      symbol : Optional[str],
                      add_amounts : Union[int, List[int]]) -> None:
        pass
    
    def withdraw_liquidity(self, 
                           symbol : Optional[str],
                           withdraw_amounts : Union[int, List[int]]) -> None:
        pass
    
    def exchange(self, 
                 sell_token_symbol : str,
                 buy_token_symbol : str,
                 sell_amount : int,
                 buy_amount : int) -> None:
        
        sell_token_id = self._get_token_id(sell_token_symbol)
        buy_token_id = self._get_token_id(buy_token_symbol)
        
        # There is no notion of address to send/receive funds to/from.
        self.token_balances[sell_token_id] -= sell_amount
        self.token_balances[buy_token_id] -= buy_amount
        
        # There is no price update. This fake pool is an arbitrage goldmine! 
        

fake_pool = FakePool(
    token_symbols = ["token 1", "token 2"],
    token_prices = [1, 2],
    token_balances = [200, 100],
    exchange_fee = 0.05
)

In [38]:
fake_pool._get_token_id("token 2")

1

In [39]:
print(fake_pool)

token 1 has balance 200 at price 1
token 2 has balance 100 at price 2


In [40]:
# For a pool with two assets I want to be observe the price impact on asset X when I add/subtract some amount of asset X. 
# I want to produce the plots of reserves vs. price and delta_reserves vs. price.

fake_pool.exchange("token 1", "token 2", 20, 10)
print(fake_pool)

token 1 has balance 180 at price 1
token 2 has balance 90 at price 2


## Curve

In [ ]:
from typing import Union, Optional, List

class CurveTricryptoPool(Pool):
    
    def __init__(self, 
                 token_symbols : List[str], 
                 token_prices : List[float],
                 token_balances : List[int] = None,
                 exchange_fee : float = 0.0,
                 pool_params : dict = {}) -> None:

        super().__init__(token_symbols, token_prices, token_balances, exchange_fee)
        self.pool_params = pool_params
        
    def tweak_price(self,
                    A_gamma : int = None,
                    _xp : list[int] = None,
                    i : int = None, 
                    p_i : int = None, 
                    new_D : int = None) -> None:
        '''
        This is a function in the real Curve smart contracts.
        https://github.com/curvefi/curve-crypto-contract/blob/master/contracts/tricrypto/CurveCryptoSwap.vy#L434
        
        :param int A_gamma 
        :param list[int] _xp
        :param int i
        :param int p_i 
        :param int new_D
        '''
        
        ### refactor of curve crypto pool tweak_price
        # TODO: read https://vyper.readthedocs.io/en/v0.2.12/built-in-functions.html#bitwise-operations
        # TODO: understand how price scales need to be reflected in this pool
        # TODO: select contract's self.state_vars needed in this classes self.state_vars
        
        pass
    
        
        
    def add_liquidity(self, 
                      symbol : Optional[str],
                      add_amounts : Union[int, List[int]]) -> None:
        pass
    
    def withdraw_liquidity(self, 
                           symbol : Optional[str],
                           withdraw_amounts : Union[int, List[int]]) -> None:
        pass
    
    def exchange(self, 
                 sell_token_symbol : str,
                 buy_token_symbol : str,
                 sell_amount : int,
                 buy_amount : int) -> None:
        
        sell_token_id = self._get_token_id(sell_token_symbol)
        buy_token_id = self._get_token_id(buy_token_symbol)
        
        # There is no notion of address to send/receive funds to/from.
        self.token_balances[sell_token_id] -= sell_amount
        self.token_balances[buy_token_id] -= buy_amount
        
        ### dynamic repegging logic ###
        # TODO: use EMA price oracle to update token_balances * token_prices
        # TODO: update invariant and price scale
        # TODO: "repeg" by adjusting price scale 
            # TODO: only repeg after measuring profit and comparing to actual profit

# TODO - Smart Contract Languages and OOP

## Curve

### TODO - Understanding Vyper
A Curve pool is a smart contract that implements the StableSwap invariant and thereby allows for the exchange of two or more tokens. Curve pool contracts are written in Vyper.  

https://curve.readthedocs.io/exchange-pools.html


Like objects in OOP, each contract written in Vyper contains state variables, functions, and common data types. 

The keywords in Vyper we want to understand and convert into OOP constructs are:
* `interface`
* `event` 
* `@view` & `@internal` & `@external`

## Uniswap

### TODO - Understanding Solidity

Uniswap pools primarily serve as automated market makers for the paired assets. Additionally, they expose price oracle data and may be used as an asset source for flash transactions.

Uniswap is written in Solidity. 

https://docs.uniswap.org/protocol/reference/core/UniswapV3Pool

https://docs.uniswap.org/protocol/reference/deployments

You can look up the address of an existing pool on [Uniswap Info](https://info.uniswap.org/#/) or by calling the [getPool](https://docs.uniswap.org/protocol/reference/core/interfaces/IUniswapV3Factory#getpool) function on the UniswapV3Factory contract.